## **Experimenting With Langchain to make simple chatbot**

### **Importing Libraries**

In [77]:
from langchain_openai import OpenAI

### **Setting up Environment Variables**

In [78]:
import os
from dotenv import load_dotenv

load_dotenv()

# Let's call environment variables
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACKING_V2"] = "true"

### **`Setting Up OpenAi llm`**

In [79]:
llm = OpenAI(
    openai_api_key=os.environ["OPENAI_API_KEY"], 
    temperature=0.6
)

#### **Let's check if defined llm is working fine**

In [80]:
text = "what is capital of Pakistan?"
print(llm.invoke(text))



The capital of Pakistan is Islamabad.


### **`Setting Up HuggingFace llm`**

In [81]:
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["HUGGINGFACE_API_TOKEN"] = "***"

In [82]:
from langchain import HuggingFaceHub

llm_huggingface = HuggingFaceHub(
    repo_id="google/flan-t5-large",
    model_kwargs={
        "temperature": 0.0,
        "max_length": 128,
    },
    huggingfacehub_api_token="***"
)

In [83]:
output = llm_huggingface.predict("what is capital city of Pakistan?")
print(output)

Karachi


### **Prompt Template**

In [84]:
from langchain.prompts import PromptTemplate

prompt_template=PromptTemplate(input_variables=['country'],
template="Tell me the capital of this {country}")

prompt_template.format(country="Pakistan")

'Tell me the capital of this Pakistan'

### **Setting up LLMChain**

In [85]:
from langchain.chains import LLMChain
chain = LLMChain(
    llm = llm,
    prompt = prompt_template
)
print(chain.run("Pakistan"))



The capital of Pakistan is Islamabad.


### **Combining multiple Chains With Simple Sequential Chain**

In [86]:
capital_template = PromptTemplate(
    input_variables=['country'],
    template="Please tell me the capital of the {country}"
)

capital_chain = LLMChain(
    llm = llm,
    prompt = capital_template
)

famous_template = PromptTemplate(
    input_variables=['capital'],
    template="Suggest me some amazing places to visit in {capital}"
)

famous_chain = LLMChain(
    llm = llm,
    prompt = famous_template
)

In [87]:
from langchain.chains import SimpleSequentialChain

chain = SimpleSequentialChain(
    chains=[capital_chain,famous_chain]
)
chain.run("Pakistan")

'\n1. Faisal Mosque - This iconic mosque is one of the largest in the world and is known for its stunning white marble architecture.\n\n2. Margalla Hills National Park - This scenic national park is located on the outskirts of Islamabad and offers breathtaking views of the city, as well as hiking and trekking opportunities.\n\n3. Lok Virsa Museum - This museum showcases the rich cultural heritage of Pakistan through its collection of traditional art, crafts, and artifacts.\n\n4. Rawal Lake - This artificial lake is a popular spot for picnics, boating, and birdwatching.\n\n5. Pakistan Monument - This monument is a symbol of national unity and features a beautiful garden, museum, and observation deck.\n\n6. Saidpur Village - This charming village is located on the outskirts of Islamabad and is known for its traditional architecture, handicrafts, and local cuisine.\n\n7. Daman-e-Koh - This viewpoint offers panoramic views of Islamabad and is a popular spot for picnics and hiking.\n\n8. Ce

### **Sequential Chain**

In [88]:
capital_template = PromptTemplate(
    input_variables=['country'],
    template="Please tell me the capital of the {country}"
)

capital_chain = LLMChain(
    llm = llm,
    prompt = capital_template,
    output_key="capital"
)

famous_template = PromptTemplate(
    input_variables=['capital'],
    template="Suggest me some amazing places to visit in {capital}"
)

famous_chain = LLMChain(
    llm = llm,
    prompt = famous_template,
    output_key="places"
)

In [89]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains=[capital_chain,famous_chain],
    input_variables=["country"], 
    output_variables=["capital","places"],   
)
chain({"country":"Pakistan"})

{'country': 'Pakistan',
 'capital': '\n\nThe capital of Pakistan is Islamabad.',
 'places': " Some amazing places to visit in Islamabad are:\n\n1. Faisal Mosque: This iconic mosque is one of the largest mosques in the world and a must-visit for its stunning architecture and peaceful atmosphere.\n\n2. Margalla Hills: These beautiful hills offer breathtaking views of the city and are perfect for hiking, picnics, and nature walks.\n\n3. Lok Virsa Museum: This museum showcases the rich cultural heritage of Pakistan and is a great place to learn about the country's history and traditions.\n\n4. Rawal Lake: This artificial lake is a popular spot for boating, picnics, and bird watching. It also has a beautiful park and a small zoo.\n\n5. Pakistan Monument: This national monument is a symbol of unity and features a museum, art gallery, and a beautiful view of the city.\n\n6. Saidpur Village: This charming village on the outskirts of Islamabad offers a glimpse into rural life and has quaint caf

## **Chatmodels with ChatOpenAI**

In [96]:
from langchain_openai import ChatOpenAI
from langchain.schema import HumanMessage,SystemMessage,AIMessage

chat_llm = ChatOpenAI(
    openai_api_key = os.environ["OPENAI_API_KEY"],
    temperature = 0.6,
    model = 'gpt-3.5-turbo'
)

In [98]:
# schema

chat_llm([
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="Tell me the capital of Pakistan?")
])

AIMessage(content='The capital of Pakistan is Islamabad.', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 24, 'total_tokens': 31}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-1bc19ab3-4796-42c6-89b4-a21adcb13ec0-0', usage_metadata={'input_tokens': 24, 'output_tokens': 7, 'total_tokens': 31})

### **Prompt Template + LLM + Output Parsers**

In [100]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import BaseOutputParser

In [102]:
class Commaseperatedoutput(BaseOutputParser):
    def parse(self, text):
        return text.strip().split(",")


template="Your are a helpful assistant. When the use given any input , you should generate 5 words synonyms in a comma seperated list."

human_template="{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system",template),
    ("human",human_template)
])

In [104]:
chain = chat_prompt | chat_llm | Commaseperatedoutput()

chain.invoke({"text" : "intellegent"})

['smart', ' clever', ' astute', ' bright', ' sharp']